In [4]:
# SVD 奇异值分解
'''
Created on Mar 8, 2018

@author: xuming
'''

'\nCreated on Mar 8, 2018\n\n@author: xuming\n'

In [5]:
import numpy as np
import numpy.linalg as la

In [12]:
def loadExData():
    dataMat = np.array(
        [[0, 0, 0, 2, 2],
         [0, 0, 0, 3, 3],
         [0, 0, 0, 1, 1],
         [1, 1, 1, 0, 0],
         [2, 2, 2, 0, 0],
         [5, 5, 5, 0, 0],
         [1, 1, 1, 0, 0]])
    return dataMat

In [11]:
def loadExData2():
    dataMat1 = np.array(
        [[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
         [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
         [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
         [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
         [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
         [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
         [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
         [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
         [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
         [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]])
    return dataMat

In [13]:
def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))

In [14]:
def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

In [ ]:
def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [1]:
def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item].A>0, \
                                      dataMat[:,j].A>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal
    
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = la.svd(dataMat)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1]#find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

def printMat(inMat, thresh=0.8):
    for i in range(32):
        for k in range(32):
            if float(inMat[i,k]) > thresh:
                print 1,
            else: print 0,
        print ''

def imgCompress(numSV=3, thresh=0.8):
    myl = []
    for line in open('0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = mat(myl)
    print "****original matrix******"
    printMat(myMat, thresh)
    U,Sigma,VT = la.svd(myMat)
    SigRecon = mat(zeros((numSV, numSV)))
    for k in range(numSV):#construct diagonal matrix from vector
        SigRecon[k,k] = Sigma[k]
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:]
    print "****reconstructed matrix using %d singular values******" % numSV
    printMat(reconMat, thresh)

SyntaxError: invalid syntax (<ipython-input-1-2452e1f32485>, line 52)